# 5. Leer datos

In [18]:
import pandas as pd
import numpy as np
from datetime import date

In [19]:
obras = pd.read_excel("./obras/plays_data.xlsx")
obras.head()

,Titulo,Teatro,Sala,Fechas,Imagen,Link,Ano_inicio,Activa
0,\n Los gestos\n,\n \n\n Teatro Valle-Inc...,\n \n Sala Grande\n ...,\n 01 DIC - 14 ENE,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/los-gestos/,2023,True
1,\n PLAY! (Nuevos Dramáticos)\n ...,\n \n\n Teatro María Guerr...,\n \n Sala de la Princesa\n...,\n 30 NOV - 20 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/play-nuevos-dr...,2023,True
2,\n Forever\n,\n \n\n Teatro María Guerrer...,\n \n Sala Grande\n \n,\n 29 NOV - 30 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/forever/,2023,True
3,\n Hedda Gabler\n,\n \n\n Teatro Valle-Inclán\...,\n \n Sala Francisco Nieva\n ...,\n 22 NOV - 30 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/hedda-gabler/,2023,True
4,\n Estación paraíso\n,\n \n\n Titerescena\n ...,\n \n Sala El Mirlo Blanco\n ...,\n 16 DIC - 17 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/estacion-paraiso/,2023,True


### Limpiar datos

1. hay caracteres "\n" (saltos de línea) entre los datos. 
2. El formato de las fechas no es estándar.

In [20]:
columns_to_clean = ['Titulo', 'Teatro', 'Sala', 'Fechas']
obras[columns_to_clean] = obras[columns_to_clean].replace('\n', '', regex=True)

# Print the cleaned DataFrame
obras.head()

,Titulo,Teatro,Sala,Fechas,Imagen,Link,Ano_inicio,Activa
0,Los gestos,Teatro Valle-Inclán ...,Sala Grande,01 DIC - 14 ENE,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/los-gestos/,2023,True
1,PLAY! (Nuevos Dramáticos),Teatro María Guerrero ...,Sala de la Princesa ...,30 NOV - 20 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/play-nuevos-dr...,2023,True
2,Forever,Teatro María Guerrero ...,Sala Grande,29 NOV - 30 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/forever/,2023,True
3,Hedda Gabler,Teatro Valle-Inclán ...,Sala Francisco Nieva,22 NOV - 30 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/hedda-gabler/,2023,True
4,Estación paraíso,Titerescena ...,Sala El Mirlo Blanco,16 DIC - 17 DIC,https://dramatico.mcu.es/wp-content/uploads/20...,https://dramatico.mcu.es/evento/estacion-paraiso/,2023,True


In [21]:
# Split "Fechas" column into two columns
obras[['Fecha_inicio_raw', 'Fecha_final_raw']] = obras['Fechas'].str.split('-', expand=True)

# Strip any leading or trailing whitespaces
obras['Fecha_inicio_raw'] = obras['Fecha_inicio_raw'].str.strip()
obras['Fecha_final_raw'] = obras['Fecha_final_raw'].str.strip()

# Map Spanish month names to their corresponding numerical values
spanish_month_names = {
    'ENE': '01', 'FEB': '02', 'MAR': '03', 'ABR': '04',
    'MAY': '05', 'JUN': '06', 'JUL': '07', 'AGO': '08',
    'SEP': '09', 'OCT': '10', 'NOV': '11', 'DIC': '12'
}

# Extract day, month, and year from "Fecha_inicio_raw"
obras[['Dia_inicio', 'Mes_inicio']] = obras['Fecha_inicio_raw'].str.split(' ', expand=True)
obras['Mes_inicio'] = obras['Mes_inicio'].map(spanish_month_names)
obras['Fecha_inicio'] = obras['Ano_inicio'].astype(str) + '/' + obras['Mes_inicio'] + '/' + obras['Dia_inicio']

# Extract day, month, and year from "Fecha_final_raw"
obras[['Dia_final', 'Mes_final']] = obras['Fecha_final_raw'].str.split(' ', expand=True)
obras['Mes_final'] = obras['Mes_final'].map(spanish_month_names)
obras['Fecha_final'] = obras['Ano_inicio'].astype(str) + '/' + obras['Mes_final'] + '/' + obras['Dia_final']

# Convert "Fecha_inicio" and "Fecha_final" to datetime format
obras['Fecha_inicio'] = pd.to_datetime(obras['Fecha_inicio'])
obras['Fecha_final'] = pd.to_datetime(obras['Fecha_final'])

# Handle cases where the play spans across the year boundary
obras.loc[obras['Fecha_final'] < obras['Fecha_inicio'], 'Ano_inicio'] += 1

# Recalculate "Fecha_final" after adjusting the year
obras['Fecha_final'] = obras['Ano_inicio'].astype(str) + '/' + obras['Mes_final'] + '/' + obras['Dia_final']

# Convert "Fecha_final" to datetime format
obras['Fecha_final'] = pd.to_datetime(obras['Fecha_final'])

# Drop unnecessary columns
obras = obras.drop(['Fechas', 'Ano_inicio', 'Fecha_inicio_raw', 'Fecha_final_raw', 'Dia_inicio', 'Mes_inicio', 'Dia_final', 'Mes_final'], axis=1)

# Print the cleaned DataFrame
print(obras.head())

                                             Titulo  \
0                            Los gestos               
1              PLAY! (Nuevos Dramáticos)              
2                                 Forever             
3                            Hedda Gabler             
4                        Estación paraíso             

                                              Teatro  \
0                          Teatro Valle-Inclán   ...   
1                        Teatro María Guerrero   ...   
2                      Teatro María Guerrero     ...   
3                      Teatro Valle-Inclán       ...   
4                      Titerescena               ...   

                                                Sala  \
0                             Sala Grande              
1                       Sala de la Princesa      ...   
2                               Sala Grande            
3                      Sala Francisco Nieva            
4                      Sala El Mirlo Blanco         

In [23]:
obras['Duracion'] = (obras['Fecha_final'] - obras['Fecha_inicio']).dt.days

print(obras["Duracion"])

0     44
1     20
2     31
3     38
4      1
      ..
94     3
95    32
96    23
97     1
98     9
Name: Duracion, Length: 99, dtype: int64


In [29]:
obras["Duracion"].describe()


                                             Titulo  \
0                            Los gestos               
1              PLAY! (Nuevos Dramáticos)              
2                                 Forever             
3                            Hedda Gabler             
4                        Estación paraíso             

                                              Teatro  \
0                          Teatro Valle-Inclán   ...   
1                        Teatro María Guerrero   ...   
2                      Teatro María Guerrero     ...   
3                      Teatro Valle-Inclán       ...   
4                      Titerescena               ...   

                                                Sala  \
0                             Sala Grande              
1                       Sala de la Princesa      ...   
2                               Sala Grande            
3                      Sala Francisco Nieva            
4                      Sala El Mirlo Blanco         

In [30]:
obras['entradas_vendidas'].describe()

count      99.000000
mean     1093.696970
std      1075.697841
min        20.000000
25%        27.500000
50%       966.000000
75%      1979.500000
max      3301.000000
Name: entradas_vendidas, dtype: float64

## Análisis de datos

In [13]:
# Tipos de las columnas
obras.dtypes

Titulo                  object
Teatro                  object
Sala                    object
Imagen                  object
Link                    object
Activa                    bool
Fecha_inicio    datetime64[ns]
Fecha_final     datetime64[ns]
dtype: object

In [14]:
# Comprobar el número de NAs

obras.isna().sum()

Titulo          0
Teatro          0
Sala            2
Imagen          0
Link            0
Activa          0
Fecha_inicio    0
Fecha_final     0
dtype: int64

## Gráficas

1. Obras en cada teatro - sala?
2. Obras en cada año


In [ ]:
np.random.normal(1500, 9.7, n_m)